На вход контроллера поступают обобщенные координаты шарниров **$q$**, обобщенные скорости шарниров **$dq$**, время **$t$** (в целом, можно выкинуть, непосредственно в контроллере оно не используется) и желаемые координаты формата **XYZQUAT**, получаемые в модуле simulator.py из координат mocap-объекта. 

Хардкодом в контроллере обозначаются желаемые скорости $\dot{q}_{des}$  и ускорения $\ddot{q}_{des}$, равные нулю (хотим, чтобы тело покоилось в конечной точке). 

Также инициализируются коэффициенты PD-регулятора $$K_p, K_d \in R^6$$ - для каждой координаты по осям и вращения вокруг осей собственные коэффициенты 

С использованием фактических обобщенных координат вычисляются все сущности динамики с помощью функции ```pin.computeAllTerms(model, data, q, dq)``` \
Желаемые координаты формата XYZQUAT переводятся в формат pinocchio, после чего с помощью функции ```pin.XYZQUATToSE3``` переводятся в формат SE3, в котором мы имеем матрицу вращения и смещения по осям координат, записываем их как $R_{des}$ и $p_{des}$ соответственно

Находим якобианы эндэффектора с учетом того, что линейные скорости нас интересуют относительно базы, а угловые - относительно эндэффектора. Для этого используем функции ```pin.getFrameJacobian``` для якобианов и ```pin.getFrameJacobianTimeVariation``` для производных якобианов по времени. Используем ```pin.LOCAL``` для работы в локальном фрейме и ```pin.LOCAL_WORLD_ALIGNED``` для работы относительно базы \
Первые три строки из LWA якобианов сочетаются со следующими тремя строками из L якобианов, получаем якобианы, с которыми работаем в дальнейшем.

Из pin.Data объекта модели можем достать положение эндэффектора и получить его смещение $p$ и поворот $R$

Находим ошибку смещения как $$p_{err} = p_{des} - p,$$ а ошибку ориентации как $$R_{err} = log(R_{des}R^T).$$
Совмещаем их в один вектор, используемый для дальнейших расчетов

Скорость изменения ошибки находим как $$\dot{err} = \dot{q}_{des} - J\dot{q}$$

После чего находим ускорение системы по формуле: 
$$a_q = J^{-1}(\ddot{q}_{des} + K_p\cdot err + K_d \cdot\dot{err} - \dot{J}\dot{q})$$

И подставляем его в классическую обратную динамику:

$$\tau = M(q)a_q + h(q,\dot{q})$$

Данный вектор обобщенных сил передаем в управление моторами.


Для получения графиков файл ```simulator.py``` был модифицирован, добавлены атрибуты класса **```Simulator```**, хранящие в массивах значения исследуемых величин, на каждом шаге симуляции в массивы добавлялись текущие значения, после чего графики сохранялись функцией ```save_plots```

Для слежения за траекторией, был модифицирован код, записывающий *desired* координаты, теперь в *desired* записывались следующие координаты: $$x_{des} = x_{mocap},$$
$$ y_{des} = 0.1sin(t) + y_{mocap},$$  
$$z_{des} = 0.1cos(t) + z_{mocap}$$
На каждой итерации обновления *desired* координат также передавалось время шага для тригонометрических функций